# Model module tutorial

This tutorial is a part of Model module guide. Here, we explore how you can use the MatlabModel wrapper to use your Matlab deep learning models in the benchmark.

In [1]:
# If you are executing this notebook at a location different than root, you need to add it to
# the path.
import sys
sys.path.insert(0,'../../')

In [2]:
# Python packages
import gc
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

In [3]:
# Matlab engine
try:
    import matlab.engine
except ImportError as err:
    print(err)

# Matlab engine
eng = matlab.engine.start_matlab()

In [4]:
# Benchmark modules
from OpenDenoising import data
from OpenDenoising import model
from OpenDenoising import evaluation

Using TensorFlow backend.


In [5]:
def display_results(clean_imgs, noisy_imgs, rest_images, name):
    """Display denoising results."""
    fig, axes = plt.subplots(5, 3, figsize=(15, 15))
    
    plt.suptitle("Denoising results using {}".format(name))

    for i in range(5):
        axes[i, 0].imshow(np.squeeze(clean_imgs[i]), cmap="gray")
        axes[i, 0].axis("off")
        axes[i, 0].set_title("Ground-Truth")

        axes[i, 1].imshow(np.squeeze(noisy_imgs[i]), cmap="gray")
        axes[i, 1].axis("off")
        axes[i, 1].set_title("Noised Image")

        axes[i, 2].imshow(np.squeeze(rest_imgs[i]), cmap="gray")
        axes[i, 2].axis("off")
        axes[i, 2].set_title("Restored Images")

In [6]:
data.download_BSDS_grayscale(output_dir="../../tmp/BSDS500/")

2019-09-12 17:31:04,191 : WARNING : OpenDenoising.data : Directory ../../tmp/BSDS500/Train/ref already exists. Download was aborted. Check if the data was already downloaded.
2019-09-12 17:31:05,543 : WARNING : OpenDenoising.data : Directory ../../tmp/BSDS500/Valid/ref already exists. Download was aborted. Check if the data was already downloaded.


In [7]:
# Validation images generator
valid_generator = data.DatasetFactory.create(path="../../tmp/BSDS500/Valid",
                                             batch_size=8,
                                             n_channels=1,
                                             noise_config={data.utils.gaussian_noise: [25]},
                                             name="BSDS_Valid")

# Matlab model <a class="anchor" id="Matlab-model"></a> 
---

# Navigation
---

* [Matlab Model](#Matlab-model)
    * [Charging a model](#matlab-charging-a-model)
    * [Training a MatlabModel](#matlab-model-training)
    * [Inference with MatlabModel](#matlab-model-inference)

## Charging a model <a class="anchor" id="matlab-charging-a-model"></a> 

To charge a Matlab model, you can either specify a Matlab function or a .mat file containing the architecture you want to train/test. In both cases, you need to specify a string.

### From a file

To charge a model from a file, you need to specify the path to the .mat file containing the model's architecture. Notice that for models that predict the residual, rather than the restored image, "__return_diff__" should be specified as True,

In [8]:
matlabModel = model.MatlabModel(return_diff=True)

In [9]:
matlabModel.charge_model(model_path="./Additional Files/Matlab Models/dncnn_matlab.mat")

After charging the model into the wrapper object, the network object will be available on Matlab's workspace. The following command prints the workspace into Jupyter's terminal window:

In [10]:
matlabModel.engine.workspace

### From a Function

To specify a model from a function, you need to specify the path to the .m file that has the function that will build your model. This string is used internally to add the .m file to the path, and the to call the function using Matlab's engine.

__Note:__ you may still pass extra arguments through kwargs, as if they were going to feed a normal Python function.

In [11]:
matlabModel2 = model.MatlabModel(return_diff=True, logdir="../../logs/Matlab")
matlabModel2.charge_model(model_function="../../OpenDenoising/model/architectures/matlab/dncnn.m")

## Inference with MatlabModel <a class="anchor" id="matlab-model-inference"></a> 

To perform inference, you may use the "\_\_call\_\_" method in MatlabModel class. This method uses the Matlab's engine to internally call "denoiseImage" matlab function, that uses the network object to denoise an input batch.

In [12]:
# Get batch from valid_generator
noisy_imgs, clean_imgs = next(valid_generator)
# Performs inference on noisy images
rest_imgs = matlabModel(noisy_imgs)

MatlabExecutionError: 
  File /usr/local/MATLAB/R2018b/toolbox/distcomp/gpu/gpuDevice.m, line 26, in gpuDevice

  File /usr/local/MATLAB/R2018b/toolbox/nnet/cnn/+nnet/+internal/+cnn/+util/isGPUCompatible.m, line 10, in isGPUCompatible

  File /usr/local/MATLAB/R2018b/toolbox/nnet/cnn/+nnet/+internal/+cnn/+util/GPUShouldBeUsed.m, line 17, in GPUShouldBeUsed

  File /usr/local/MATLAB/R2018b/toolbox/nnet/cnn/SeriesNetwork.m, line 866, in SeriesNetwork.getPredictionEnvironment

  File /usr/local/MATLAB/R2018b/toolbox/nnet/cnn/SeriesNetwork.m, line 771, in SeriesNetwork.activations

  File /usr/local/MATLAB/R2018b/toolbox/images/images/denoiseImage.m, line 73, in denoiseImage
An unexpected error occurred during CUDA execution. The CUDA error was:
out of memory


In [ ]:
display_results(clean_imgs, noisy_imgs, noisy_imgs - rest_imgs, str(matlabModel))

## Training a MatlabModel <a class="anchor" id="matlab-model-training"></a> 

To train a MatlabModel, you need to specify a training (and possibly a validation) dataset through a string. This string correspond to the name of the dataset in Matlab's workspace.

To create the dataset in the workspace, you can use the classes 'imageDatastore', 'CleanMatlabDataset' and 'FullMatlabDataset', which are Matlab classes for generating data to train Deep Learning models.

### Using a CleanDataset

As in the case of Python's CleanDatasetGenerator, to specify a Clean Dataset using Matlab you need to specify the noising function, called noiseFcn. This function should be specified as a string, that has the __lambda signature__ on it.

For instance, if you want to use Gaussia noise on your dataset, you need to specify:

noiseFcn = "@(I) imnoise(I, 'gaussian', 0, 25/255)".

For more complex kinds of functions, you can implement it as a .m function, and specify its arguments via the same strategy.

__Note:__ You should make sure that "./OpenDenoising/data/" folder is on Matlab's path (add it to pathdef.m).

In [14]:
dataset_train_wrapper = data.MatlabDatasetWrapper(matlabModel2.engine, images_path="../../tmp/BSDS500/Train/ref",
                                                         partition="Train")
dataset_train_wrapper()

In [15]:
dataset_valid_wrapper = data.MatlabDatasetWrapper(matlabModel2.engine, images_path="../../tmp/BSDS500/Valid/ref",
                                                         partition="Valid")
dataset_valid_wrapper()

In [16]:
matlabModel2.engine.workspace

In [17]:
matlabModel2.train(train_generator="imds_Train_noise", valid_generator="imds_Valid_noise")

opts = trainingOptions('adam', 'Plots', 'training-progress','Verbose', true,'VerboseFrequency', 500,'MaxEpochs', 250,'Shuffle', 'every-epoch','ValidationData', valid_imds,'CheckpointPath', '../../OpenDenoising/training_logs/Matlab');
